In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [3]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium["EX_fru_e"]= 0
medium["EX_co2_e"]= 1000
medium["EX_co_e"]= 10
medium["EX_h2_e"]= 10
Cljung_Model.medium = medium
solution = Cljung_Model.optimize()

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


# Determine minimal set of reactions for acetogenesis:

In [4]:
Cljung_Model.objective = "ATPM"
solution = Cljung_Model.optimize()

In [5]:
solution.fluxes[solution.fluxes != 0]

EX_h2_e    -10.0
EX_ac_e      5.0
EX_co_e    -10.0
ACt2r       -5.0
ACKr        -5.0
ATPM         5.0
EX_h_e       5.0
COt        -10.0
CODH4       10.0
CODH_ACS     5.0
H2td       -10.0
FDH7         5.0
FTHFLi       5.0
METR         5.0
HYDFDN2r     5.0
MTHFD       -5.0
MTHFC       -5.0
MTHFR5       5.0
PTAr         5.0
RNF         10.0
ATPS4r       5.0
Name: fluxes, dtype: float64

In [31]:
Builder(model = Cljung_Model, map_json = join(data_dir,'Models/WL_core.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': -10.0, 'EX_h2o_e': 2.4999999999999947, 'EX_co2_e': 2.5, 'EX_co_e': -10.0, 'E…

In [8]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt2r,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,
#Energy:
Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.HYDFDi,
Cljung_Model.reactions.HYDFDN2r,
Cljung_Model.reactions.ATPS4r,
Cljung_Model.reactions.ATPM,
Cljung_Model.reactions.RNF, #Add this reaction in Escher
#WL pathway
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2y,
Cljung_Model.reactions.ALCD2x,
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

In [42]:
WLmodel.objective = "ATPM"
WLmodel.optimize()

0.0

In [45]:
WLmodel.objective = "ATPM"
solution = WLmodel.optimize()
print("Flux through objective function: ",WLmodel.objective.expression, " = ",round(WLmodel.slim_optimize(),1))
Builder(model = WLmodel, map_json = join(data_dir,'Models/WL_core.json'), reaction_data = solution.fluxes.to_dict())

Flux through objective function:  1.0*ATPM - 1.0*ATPM_reverse_5b752  =  4.2


Builder(reaction_data={'EX_h2_e': -10.0, 'EX_h2o_e': 0.0, 'EX_co2_e': 3.3333333333333326, 'EX_co_e': -10.0, 'E…

# Add Acetone production pathway

In [12]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [13]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: 1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{'H': 2.0}
(0.0, 1000.0)
aacoa_c + ac_c --> acac_c + accoa_c + h_c


In [14]:
thl = Reaction("THL")
thl.name = "acetyl-CoA C-acetyltransferase"

thl.add_metabolites({
    WLmodelActn.metabolites.accoa_c: -2.0,     #Acetyl-coa
    WLmodelActn.metabolites.h_c: -1.0,
    WLmodelActn.metabolites.aacoa_c: 1.0,     #Acetoacetyl-CoA
    WLmodelActn.metabolites.coa_c: 1.0
})
print(thl.check_mass_balance())    #ensure correct mass balance
print(thl.reaction)                #Ensure correct reaction

{'H': -1.0}
2.0 accoa_c + h_c --> aacoa_c + coa_c


In [15]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{'H': -1.0}
acac_c + h_c --> act_c + co2_c


In [16]:
actT = Reaction("actT")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,actT
Name,Acetone transporter
Memory address,0x07fd565b916a0
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07fd565bc64a8
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
WLmodelActn.add_reactions([ctfAB_ac, thl, adc, actT, act_ex])

# test 

In [26]:
WLmodelActn.objective = "EX_act_e"
solution = WLmodelActn.optimize()
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/AcetonePathway2.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': -10.0, 'EX_h2o_e': 2.4999999999999947, 'EX_co2_e': 2.5, 'EX_co_e': -10.0, 'E…

In [22]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/WL_core.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': -1.08, 'EX_h2o_e': 0.0, 'EX_co2_e': 0.5400000000000005, 'EX_co_e': -1.440000…

Builder(model = Moorella_Model)#, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'))

Moorella_Model.metabolites.get_by_id("10fthf_c")

Moorella_Model.metabolites.methf_c

Moorella_Model.metabolites.mlthf_c

Moorella_Model.reactions.TMDS

CO2t
CODH_ACS
PFOR
Moorella_Model.reactions.CODH

#Transport & Exchange:
Moorella_Model.reactions.EX_co_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_co2_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_h2_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_ac_LPAREN_e_RPAREN
Moorella_Model.reactions.COt
Moorella_Model.reactions.CO2t
Moorella_Model.reactions.H2td
Moorella_Model.reactions.ACt2r

#Energy:
Moorella_Model.reactions.HYDFDN
Moorella_Model.reactions.CODH

#Eastern branch:
Moorella_Model.reactions.FDH

#Western branch:
Moorella_Model.reactions.CODH_ACS


WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([Moorella_Model.reactions.CODH])

WLmodel = Model('Wood_Ljungdahl_Core')

reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default